In [1]:
from os import listdir
from os.path import splitext
from os.path import join

import skvideo
skvideo.setFFmpegPath(r'C:\Users\ion\anaconda3\envs\tf-gpu\Lib\site-packages\ffmpeg-4.3.1-2020-10-01-full_build\bin')
import skvideo.io
import json
import cv2

## Explanation of different datasets
The Aff-Wild2 dataset is used for training, validation and testing. While the AFEW 7.0 dataset is only used for testing. 

#### AFEW 7.0
In this dataset each video shows only one facial expression displayed, therefore each video is placed in a folder, where the folder name represents the facial expression class. The test dataset videos classes are not given, therefore only the train+validation set will be combined to use for final testing results of the model.

In [2]:
# AFEW 7.0  = AF7
# filenames = FN

### AFEW 7.0 ###
# Directories for labels/annotations for each set
AF7_dir_videos = r"D:\AFEW 7.0 Dataset\Val+train_AFEW"
AF7_dir_labels = listdir(AF7_dir_videos) 

AF7_labeled_videos = {}
for label in AF7_dir_labels:
    videos = listdir(join(AF7_dir_videos, label))
    AF7_labeled_videos[label] = videos

### AFEW 7.0 Dataset Size ###

In [9]:
# Size of each set
AF7_all_videos = [j for i in AF7_dir_labels for j in AF7_labeled_videos.get(i)]

print(f'The training+validation set contains: {len(AF7_all_videos)} videos')

The training+validation set contains 1156 videos


### Sequence length of each video ###

There are some issues with AFEW 7.0 dataset, most of the files show that they only have 2 frames by using SKvideo, with opencv though with a manual implementation it will return 3 frames. However for some videos the results are correct.

### NEEDS FIX ###

In [63]:
# Create dictionary of sequence length
AF7_seqlength = {}

# Save the shape information of the array for each file in the previously made dictionary
for label in AF7_labeled_videos.keys():
    videos = AF7_labeled_videos.get(label)
    for video in videos:  
        cache = skvideo.io.vread(join(AF7_dir_videos,join(label,video)))
        AF7_seqlength[video] = cache.shape

In [83]:
AF7_seqlength

{'000046280.avi': (2, 576, 720, 3),
 '000149120.avi': (68, 576, 720, 3),
 '000223480.avi': (2, 576, 720, 3),
 '000316560.avi': (51, 576, 720, 3),
 '000429620.avi': (2, 576, 720, 3),
 '000450454.avi': (2, 576, 720, 3),
 '000505440.avi': (70, 576, 720, 3),
 '000550480.avi': (2, 576, 720, 3),
 '000551374.avi': (2, 576, 720, 3),
 '000746734.avi': (2, 576, 720, 3),
 '000800760.avi': (2, 576, 720, 3),
 '000846120.avi': (1, 576, 720, 3),
 '000929080.avi': (2, 576, 720, 3),
 '000947327.avi': (2, 576, 720, 3),
 '001108440.avi': (2, 576, 720, 3),
 '001247040.avi': (2, 576, 720, 3),
 '001419640.avi': (70, 576, 720, 3),
 '001454480.avi': (2, 576, 720, 3),
 '001549894.avi': (2, 576, 720, 3),
 '001702480.avi': (35, 576, 720, 3),
 '001809774.avi': (2, 576, 720, 3),
 '001855040.avi': (2, 576, 720, 3),
 '001940720.avi': (2, 576, 720, 3),
 '001943720.avi': (2, 576, 720, 3),
 '002014840.avi': (2, 576, 720, 3),
 '002033320.avi': (2, 576, 720, 3),
 '002137000.avi': (2, 576, 720, 3),
 '002336640.avi': (65, 

In [71]:
## https://www.pyimagesearch.com/2017/01/09/count-the-total-number-of-frames-in-a-video-with-opencv-and-python/

## Count frames manual with opencv
def count_frames_manual(video):
    # initialize the total number of frames read
    total = 0
    # loop over the frames of the video
    while True:
        # grab the current frame
        (grabbed, frame) = video.read()
        
        # check to see if we have reached the end of the
        # video
        if not grabbed:
            break
        # increment the total number of frames read
        total += 1
    # return the total number of frames in the video file
    return total

In [ ]:
# Create dictionary of sequence length of each video
AF7_seqlength_V2 = {}

for label in AF7_labeled_videos.keys():
    videos = AF7_labeled_videos.get(label)
    for video in videos:  
        metadata = skvideo.io.ffprobe(join(AF7_dir_videos,join(label,video)))
        frames = count_frames_manual(cv2.VideoCapture(join(AF7_dir_videos,join(label, video))))
        
        width = int(metadata.get('video').get('@width'))
        height = int(metadata.get('video').get('@height'))
        AF7_seqlength_V2[video] = [frames, height, width, 3]

# Save json file with all video's names and shapes = (n_frames, width, height, colour channels)
# with open('AF7_video_shapes.json', 'w') as fp:
#     json.dump(AF7_seqlength, fp)

In [81]:
AF7_seqlength_V2

{'000046280.avi': [3, 576, 720, 3],
 '000149120.avi': [68, 576, 720, 3],
 '000223480.avi': [3, 576, 720, 3],
 '000316560.avi': [51, 576, 720, 3],
 '000429620.avi': [3, 576, 720, 3],
 '000450454.avi': [3, 576, 720, 3],
 '000505440.avi': [70, 576, 720, 3],
 '000550480.avi': [3, 576, 720, 3],
 '000551374.avi': [3, 576, 720, 3],
 '000746734.avi': [3, 576, 720, 3],
 '000800760.avi': [3, 576, 720, 3],
 '000846120.avi': [2, 576, 720, 3],
 '000929080.avi': [3, 576, 720, 3],
 '000947327.avi': [3, 576, 720, 3],
 '001108440.avi': [3, 576, 720, 3],
 '001247040.avi': [3, 576, 720, 3],
 '001419640.avi': [70, 576, 720, 3],
 '001454480.avi': [3, 576, 720, 3],
 '001549894.avi': [3, 576, 720, 3],
 '001702480.avi': [35, 576, 720, 3],
 '001809774.avi': [3, 576, 720, 3],
 '001855040.avi': [3, 576, 720, 3],
 '001940720.avi': [3, 576, 720, 3],
 '001943720.avi': [3, 576, 720, 3],
 '002014840.avi': [3, 576, 720, 3],
 '002033320.avi': [3, 576, 720, 3],
 '002137000.avi': [3, 576, 720, 3],
 '002336640.avi': [65, 